In [4]:
import pandas as pd
import tiktoken
import os

# Tentukan model yang akan Anda gunakan. Ini penting karena beda model, beda cara tokenisasi.
# "cl100k_base" adalah encoding yang digunakan oleh model gpt-4, gpt-3.5-turbo, dan text-embedding-ada-002
try:
    encoding = tiktoken.get_encoding("cl100k_base")
except Exception as e:
    print(f"Error saat memuat encoding tiktoken: {e}")
    exit()

def hitung_token_chatgpt(teks):
    """
    Fungsi untuk menghitung jumlah token dari sebuah teks menggunakan tokenizer resmi OpenAI.
    """
    if not isinstance(teks, str):
        return 0
    # Encode teks menjadi nomor token, lalu hitung jumlahnya
    return len(encoding.encode(teks))


# Masukkan path menuju file input
NAMA_FILE_INPUT = 'dataset_baru.csv'
TOKEN_PROMPT_PERINTAH = 430  # Definisikan token prompt sebagai variabel

# --- Mulai Proses Utama ---
try:
    print(f"\nMemuat dataset dari '{NAMA_FILE_INPUT}'...")
    df = pd.read_csv(NAMA_FILE_INPUT)

    # 1. Hitung token menggunakan TIKTOKEN dan tambahkan ke kolom baru
    print("Langkah 1: Menghitung token menggunakan tokenizer 'cl100k_base' (untuk model GPT-3.5/4)...")
    df['jumlah_token_chatgpt'] = df['isi_berita'].apply(hitung_token_chatgpt)
    print(df[['judul', 'jumlah_token_chatgpt']].head())

    print(f"  - Menambahkan {TOKEN_PROMPT_PERINTAH} token prompt ke setiap baris...")
    # Tambahkan jumlah token prompt ke setiap baris secara efisien
    df['jumlah_token_chatgpt'] += TOKEN_PROMPT_PERINTAH
    print("Penghitungan token selesai.")

    # 2. Bagi dataset menjadi beberapa batch berdasarkan jumlah token
    print("\nLangkah 2: Membagi dataset menjadi batch dengan batas token 2.0M...")
    TOKEN_LIMIT = 2_000_000
    batches = []
    current_batch_start_index = 0
    current_token_count = 0

    for index, row in df.iterrows():
        row_token_count = row['jumlah_token_chatgpt']
        
        if current_token_count > 0 and (current_token_count + row_token_count) > TOKEN_LIMIT:
            batch_df = df.iloc[current_batch_start_index:index]
            batches.append(batch_df)
            current_batch_start_index = index
            current_token_count = 0

        current_token_count += row_token_count

    # Simpan batch terakhir
    last_batch_df = df.iloc[current_batch_start_index:]
    if not last_batch_df.empty:
        batches.append(last_batch_df)

    print(f"Dataset berhasil dibagi menjadi {len(batches)} batch.")

    # 3. Outputkan setiap batch ke file CSV baru
    print("\nLangkah 3: Menyimpan setiap batch ke file CSV terpisah...")
    output_dir = "output_batches_chatgpt"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i, batch_df in enumerate(batches):
        nama_file_output = os.path.join(output_dir, f'dataset_part_{i+1}.csv')
        batch_df.to_csv(nama_file_output, index=False)
        total_tokens_in_batch = batch_df['jumlah_token_chatgpt'].sum()
        print(f"  -> Batch {i+1} disimpan ke '{nama_file_output}' ({len(batch_df)} baris, Total Token: {total_tokens_in_batch:n})")

    print("\nProses selesai!")

except FileNotFoundError:
    print(f"Error: File '{NAMA_FILE_INPUT}' tidak ditemukan.")
except KeyError:
    print(f"Error: Dataset Anda tidak memiliki kolom 'isi_berita'.")


Memuat dataset dari 'dataset_baru.csv'...
Langkah 1: Menghitung token menggunakan tokenizer 'cl100k_base' (untuk model GPT-3.5/4)...
                                               judul  jumlah_token_chatgpt
0  Kisah 'Kuli Bangunan' Bawa UMKM Bengkulu Naik ...                  2290
1  Gunung Raung Erupsi, Kolom Abu Capai 2 Ribu Meter                   489
2  Jorge Martin Dirumorkan Gabung Honda di MotoGP...                   399
3  PKS Minta Pemerintah Siapkan Skenario Darurat ...                   694
4  Malam Ini Unduh Mantu Besar dan Megah Al Ghaza...                   759
  - Menambahkan 430 token prompt ke setiap baris...
Penghitungan token selesai.

Langkah 2: Membagi dataset menjadi batch dengan batas token 2.0M...
Dataset berhasil dibagi menjadi 1 batch.

Langkah 3: Menyimpan setiap batch ke file CSV terpisah...
  -> Batch 1 disimpan ke 'output_batches_chatgpt/dataset_part_1.csv' (1460 baris, Total Token: 1923153)

Proses selesai!
